In [ ]:
import sys
sys.path.append('/home/tony/workspace/python/grad/PathPred/ptools')
sys.path = list(set(sys.path))

In [ ]:
import os
from biocyc_facade.pgdb import ImportFromBiocyc, Pgdb, Dat
from biocyc_facade.models.sql import TraceStep

DB_DIR = '/home/tony/workspace/python/grad/PathPred/ptools/data/pgdbs'
ECO = f'{DB_DIR}/eco'

# os.system(f'rm {ECO}.pgdb')
# edb = ImportFromBiocyc(ECO, '/home/tony/workspace/python/grad/resources/eco_meta_cyc_25-5/ecoli/25.1/data')

edb = Pgdb(ECO)
print(edb.GetInfo())

i = 0
# for x in edb._cur.execute("SELECT a.s_key, a.p_key FROM (si AS a) WHERE a.index_name='DBLINKS'"):
# for x in edb._cur.execute("SELECT distinct table_name from si"):
#     print(x)
#     if i > 5:
#         break
#     i+=1

# t = edb.Trace('P-DBLINKS', 'W-COMMON-NAME')
t = edb.Trace('P-DBLINKS', Dat.PROTEINS)
for tt in t:
    print(tt)
    if i > 5:
        break
    i += 1

In [ ]:
_, links, rev_links = t

In [ ]:

# Trace('P-DBLINKS', Dat.PATHWAYS)
Trace('P-DBLINKS', )

In [ ]:
x = edb.ListEntries(Dat.GENES)
x[:5]


In [ ]:
x = edb.GetEntry(Dat.GENES, 'EG10002')
x

In [ ]:
FWD, REV = True, False
method = [
    (REV, 'DBLINKS', 'proteins_dat'),
    (REV, 'ENZYME', 'enzrxns_dat'),
    (FWD, 'REACTION', 'enzrxns_dat'),
    # (REV, 'REACTION-LIST', 'pathways_dat'),
    (FWD, 'EC-NUMBER', 'reactions_dat'),
]

trace = edb._database.Trace([
    TraceStep(REV, 'DBLINKS', 'proteins_dat'),
    TraceStep(REV, 'ENZYME', 'enzrxns_dat'),
    TraceStep(FWD, 'REACTION', 'enzrxns_dat'),
    TraceStep(REV, 'REACTION-LIST', 'pathways_dat'),
])

print(len(trace))
for s, e in trace[:15]:
    print(s, " --- ", e)

In [ ]:
cur = edb._database._cur
i = 0
# p -> er
# for x in cur.execute(f"""select a, b from \
#     (select x.s_key as a, x.p_key as b from secondary_index as x where x.table_name='enzrxns_dat')"""):
# p -> r 
# for x in cur.execute(f""" \
# select x.s_key, y.s_key from \
#     (si as x inner join si as y on x.p_key=y.p_key) \
# where x.index_name='ENZYME'"""):

# p -> w
# and y.index_name='REACTION' 
# and z.index_name='REACTION-LIST'
for x in cur.execute(f"""select x.s_key, x.table_name, z.p_key from \
    (si as x inner join ( \
        si as y inner join si as z on y.s_key=z.s_key \
    ) on x.p_key=y.p_key) where x.index_name='ENZYME' and y.index_name='REACTION'  and z.index_name='REACTION-LIST'"""):
    print(x)
    if i>10: break
    i+=1

In [ ]:
FWD, REV = True, False
method = [
    (REV, 'ENZYME', 'enzrxns_dat'),
    (FWD, 'REACTION', 'enzrxns_dat'),
    # (REV, 'REACTION-LIST', 'pathways_dat'),
    (FWD, 'EC-NUMBER', 'reactions_dat'),
]

def conv(i, base=26):
    off = 97
    c = lambda x: chr(x+off-1) if x != 0 else 'z'
    if i > base-1:
        next = (i//base)*base
        rem = i - next
        print(next, rem)
        return f'{conv(next//base)}{c(rem)}'
    else:
        return f'{c(i)}'

def makeSql(methods):
    pk = 'p_key'
    sk = 's_key'
    def recurse(m, i):
        fwd, key, table = m[0]
        x = conv(i)

        if len(m) == 1:
            return f"si AS {x}", f"{x}.index_name='{key}' and {x}.table_name='{table}'"

        f2, k2, t2 = m[1]
        y = conv(i+1)
        link = f"{x}.{sk if fwd else pk}={y}.{pk if f2 else sk}"
        if len(m) == 2:
            joins = f"si AS {x} INNER JOIN si AS {y} ON {link}"
            where = f"{x}.index_name='{key}' AND {x}.table_name='{table}' AND {y}.index_name='{k2}' AND {y}.table_name='{t2}'"
        else:
            pjoins, pwhere = recurse(m[1:], i+1)
            joins = f"si AS {x} INNER JOIN ({pjoins}) ON {link}"
            where = f"{x}.index_name='{key}' AND {x}.table_name='{table}' AND {pwhere}"
        return joins, where

    j, w = recurse(methods, 1)
    ka = pk if methods[0][0] else sk
    kb = pk if not methods[-1][0] else sk
    return f"SELECT {conv(1)}.{ka}, {conv(len(methods))}.{kb} FROM ({j}) WHERE {w}"

# sql = ''
# for dir, key, table in method:
#     sql = f""

x = makeSql(method)
print(x)

In [ ]:
# p -> r 
# for x in cur.execute(f""" \
# select x.s_key, y.s_key from \
#     (si as x inner join si as y on x.p_key=y.p_key) \
# where x.index_name='ENZYME'"""):

In [ ]:
FWD, REV = True, False
method = [
    (REV, 'DBLINKS', 'proteins_dat'),
    (REV, 'ENZYME', 'enzrxns_dat'),
    (FWD, 'REACTION', 'enzrxns_dat'),
    # (REV, 'REACTION-LIST', 'pathways_dat'),
    (FWD, 'EC-NUMBER', 'reactions_dat'),
]

sql = makeSql(method[0:3])
print(sql)
sql += "and a.s_key LIKE 'UNIPROT%'"
i = 0
for x in cur.execute(sql):
    print(x)
    if i>10: break
    i+=1

In [ ]:
for k, v in edb.GetDat(Dat.PATHWAYS).items():
    t = 'REACTION-LIST'
    if t in v:
        print(k)
        print(v[t])
        break

In [ ]:
edb.GetDat(Dat.REACTIONS)['DARAB5PISOM-RXN']

In [ ]:
prm = edb.GetMapping(Mapping.RXN_PATHWAY)
for k, v in prm.items():
    print(k)
    print(v)
    break

In [ ]:
dat = edb.GetDat(Dat.REACTIONS)
for k, v in dat.items():
    if 'COMMON-NAME' in v:
        print(v['COMMON-NAME'])
        print(k)
        print(v)
        break

In [ ]:
dat['RXN0-1134']

In [ ]:
pdat = edb.GetDat(Dat.ENZRXNS)
for k, v in pdat.items():
    t = 'ENZYME'
    if t in v:
        print(k)
        print(v[t])
        print(v)
        break

In [ ]:
edb.GetDatFields(Dat.PROTEINS)

In [ ]:
from scripts.pathologic import Entry

reactions = edb.GetDat(Dat.ENZRXNS)

entries = []
others = [
    ('EC-NUMBER', 'EC'),
    ('GO-TERMS', 'GO'),
]
for k, v in reactions.items():
    e = Entry(v['COMMON-NAME'])
    break

In [ ]:
cur = edb._database._cur

# edb.registry.GetTable('json_keys_of_dats').Select(['key'], f"where table_name='{0}'")
# for x in cur.execute(f"select * from json_keys_of_dats where key='{Dat.PROTEINS.table_name}'"):
#     print(x)
for x in cur.execute(f"select id, found_in from enzrxn_reaction_map where found_in='FORMATEDEHYDROG-RXN'"):
    print(x)

In [ ]:
from enum import Enum

class Mapping(Enum):
    DB_PROT =       'proteins_dat',    'dblinks_proteins_map', 'DBLINKS',              'proteins_dat'
    PROT_ENZRXN =   'enzrxns_dat',     'protein_enzrxn_map',   'ENZYME',               'proteins_dat'
    ENZRXN_RXN =    'reactions_dat',   'enzrxn_reaction_map',  'ENZYMATIC-REACTION',   'enzrxns_dat'
    RXN_PATHWAY =   'pathways_dat',    'reaction_pathway_map', 'REACTION-LIST',        'reactions_dat'
    def __init__(self, to: str, table: str, via: str, frm: str) -> None:
        super().__init__()
        self.table_name = table
        self.map_from = frm
        self.map_to = to
        self.map_key = via 

for x in Mapping:
    print(x.value)